# Helpers

In [ ]:
library(dplyr)
library(ggplot2)
library(scales)
library(grid)
library(ggthemes)
library(gridExtra)
library(xtable)
library(ggplotify)
library(reshape2)
library(purrr)
library(latex2exp)
library(flextable)
library("ggpubr")

base.dir = "plots/"

In [ ]:
# install.packages("extrafont")
library(extrafont)
# font_import()
loadfonts(device="postscript")
loadfonts(device="pdf")
# `/usr/local/share/fonts`

# fonttable()
# names(pdfFonts())
# library(fontcm)
# names(postscriptFonts())
# names(pdfFonts())

In [3]:
ThemePublication <- function(base_size = 14, base_family = "Helvetica") {
      theme_foundation(base_size = base_size, base_family = base_family) +
      # theme_bw() +
      theme(plot.title = element_text(
                  # face = "bold", size = rel(1.2), hjust = 0.5
                  size = rel(1.2), hjust = 0.5, family= base_family, colour="#262626", vjust=3
            ),
            text = element_text(family=base_family, size=14, colour="#262626"),
            panel.background = element_rect(colour = NA),
            plot.background = element_rect(colour = NA),
            panel.border = element_rect(colour = "#d2d3d5"),
            # axis.title = element_text(face = "bold", size = rel(1)),
            axis.title = element_text(size = rel(1), colour="#262626"),
            axis.title.y = element_text(angle = 90, vjust = 2, colour="#262626"),
            axis.title.x = element_text(vjust = -0.2, colour="#262626"),
            axis.text = element_text(colour="#262626"), 
            # axis.line = element_line(colour = "#595757"),
            axis.line = element_line(colour = "#262626"),
            axis.ticks = element_line(colour="#262626"),
            panel.grid.major = element_line(colour = "#d2d3d5"),
            panel.grid.minor = element_blank(),
            legend.key = element_rect(colour = NA),
            legend.position = "bottom",
            legend.direction = "horizontal",
            legend.key.size= unit(0.5, "cm"),
            legend.spacing = unit(0, "cm"),
            legend.title = element_text(face = "italic", colour="#262626"),
            plot.margin = unit(c(10,5,5,5),"mm"),
            # strip.background = element_rect(
            #       colour = "#d2d3d5", fill = "#d2d3d5"
            # ),
            strip.background = element_blank(),
            # strip.text = element_text(face = "bold")
            strip.text = element_blank()#element_text(colour="#262626")
            # #f0f0f0
      )
}

ScalePublication <- function(ngroups = 3, ...){
      # colours <- hue_pal()(ngroups)
      colours <- c('#4c72b0', '#dd8452', '#55a868', '#c44e52', '#8172b3', '#937860', '#da8bc3', '#8c8c8c', '#ccb974', '#64b5cd')[seq(ngroups)]
      # discrete_scale("color", "Publication", manual_pal(values = colours), ...)
      discrete_scale(
            "fill", "Publication", manual_pal(values = alpha(colours, 0.4)), ...
      )
}
ScalePublication2 <- function(ngroups = 3, ...){
      # colours <- hue_pal()(ngroups)
      colours <- c('#4c72b0', '#dd8452', '#55a868', '#c44e52', '#8172b3', '#937860', '#da8bc3', '#8c8c8c', '#ccb974', '#64b5cd')[seq(ngroups)]
      discrete_scale("color", "Publication", manual_pal(values = colours), ...)
}

ScalePublication3 <- function(remove, ...){
      # colours <- hue_pal()(ngroups)
      colours <- c('#4c72b0', '#dd8452', '#55a868', '#c44e52', '#8172b3', '#937860', '#da8bc3', '#8c8c8c', '#ccb974', '#64b5cd')[-1 - remove]
      # discrete_scale("color", "Publication", manual_pal(values = colours), ...)
      discrete_scale(
            "fill", "Publication", manual_pal(values = alpha(colours, 0.7)), ...
      )
}
ScalePublication4 <- function(remove, ...){
      # colours <- hue_pal()(ngroups)
      colours <- c('#4c72b0', '#dd8452', '#55a868', '#c44e52', '#8172b3', '#937860', '#da8bc3', '#8c8c8c', '#ccb974', '#64b5cd')[-1 - remove]
      # discrete_scale("color", "Publication", manual_pal(values = colours), ...)
      discrete_scale(
            "colour", "Publication", manual_pal(values = colours), ...
      )
}

TransparentBackground <- function(remove.grid = TRUE,...){
      #https://www.r-bloggers.com/2022/07/ggplot2-transparent-background-quick-guide/
      a <- theme(
            panel.background = element_rect(fill = 'transparent'), #transparent panel bg
            plot.background = element_rect(fill = 'transparent', color=NA), #transparent plot bg
            # legend.background = element_rect(fill='transparent'), #transparent legend bg
            # legend.box.background = element_rect(fill='transparent') #transparent legend panel
            legend.background = element_blank(),
            legend.key = element_blank()
      )
      if (remove.grid) {
            a <- a + theme(
                  panel.grid.major = element_blank(), #remove major gridlines
                  panel.grid.minor = element_blank() #remove minor gridlines
            )
      }

      a
}

# https://rpubs.com/Koundy/71792
# http://www.sthda.com/english/wiki/ggplot2-box-plot-quick-start-guide-r-software-and-data-visualization
# https://stackoverflow.com/questions/14604439/plot-multiple-boxplot-in-one-graph
# https://r-graph-gallery.com/ggplot2-color.html
# http://rstudio-pubs-static.s3.amazonaws.com/5312_98fc1aba2d5740dd849a5ab797cc2c8d.html

In [4]:
BoxPlot <- function(
    data,
    plot.title = "",
    legend.title = "Experiment",
    theme.publication = TRUE,
    lbound = .7,
    ngroups = 3,
    aspect.ratio = 8,
    cossim.values = NA,
    labels = NA
){
    # options(repr.plot.width = 10, repr.plot.height = 10)

    legend.title.format <- paste0(legend.title, "  - ")

    boxplot <- ggplot(
        data = data,
        mapping = aes(
            x = normal_class,
            y = best_val_auc,
            group = interaction(normal_class, experiment),
        ),
    ) +
    geom_boxplot(aes(fill = experiment, color = experiment), lwd = .4)
    # geom_dotplot(
    #     aes(group=1),
    #     data=data.frame(
    #         best_val_auc=c(.9, .8, .7, .8, .9, .8, .7, .8, .9, .8),
    #         normal_class=c(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
    #     ),
    #     binwidth=1,
    #     dotsize=.01,
    #     binaxis="y",
    #     # stackdir="center",
    #     color='#c44e52',
    #     fill='#c44e52',
    # ) +
    # Group drawings per normal class
    if (!(length(cossim.values) == 10)) {
        boxplot <- boxplot + facet_wrap(~normal_class, scales = "free_x", nrow = 1)
    }

    # Rename Axes
    boxplot <- boxplot + ylab("Best AUC") +
    xlab("Normal Class") +

    # Further Styling
    scale_x_continuous(breaks = 0:9) +
    # https://ggplot2-book.org/scales-guides.html
    labs(fill = legend.title.format, colour = legend.title.format) +
    scale_y_continuous(expand=c(0, .001), limits=c(lbound, .999)) +
    # ylim(lbound, 1) +
    ggtitle(plot.title)
    
    if (theme.publication) {
        boxplot <- boxplot + ThemePublication(base_family="LM Roman 10")
        boxplot <- boxplot + TransparentBackground(!(length(cossim.values) == 10))
            # theme_bw()
            # ThemePublication() +
    }
    boxplot <- boxplot + ScalePublication(ngroups = ngroups, labels=labels) +
                ScalePublication2(ngroups = ngroups, labels=labels)

    # boxplot <- boxplot + theme(aspect.ratio = aspect.ratio) +
    # stat_summary(fun = median, geom = "line", aes(group=1), col="red")
    if (length(cossim.values) == 10) {
        boxplot <- boxplot + geom_line(
            aes(group=1),
            data=data.frame(
                best_val_auc=cossim.values,
                normal_class=c(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
            ),
            color='#c44e52'
        )
    }

    boxplot
}

TablePlot <- function(data, plot.title = ""){
    options(repr.plot.width = 10, repr.plot.height = 8, digits = 3)
    table <- data %>%
             group_by(experiment) %>%
             summarise_at(vars(best_val_auc), list(Mean = mean, Std = sd)) %>%
             as.data.frame()

    colnames(table)[1] <- ""

    table.grob <- tableGrob(
        d = table,
        rows = NULL,
        widths=unit(rep(1 / ncol(table), ncol(table)), "npc") - unit(1, "line"),
        heights=unit(rep(1 / nrow(table) / 1.5, nrow(table)), "npc") - unit(1, "line"),
        theme=ttheme_default(
            # padding = unit(c(4, 4), "mm")
            core = list(fg_params=list(cex = 2.0)),
            colhead = list(fg_params=list(cex = 2.0)),
            # rowhead = list(fg_params=list(cex = 1.0))
        )
    )
    table.grob <- arrangeGrob(table.grob, top=plot.title)
    table.plot <- as.ggplot(table.grob)
    list(data.frame = table, plot = table.plot)
}

ClassBoxPlot <- function(
    data,
    normal.class,
    plot.title = "",
    legend.title = "Experiment",
    theme.publication = TRUE,
    lbound = .7,
    ngroups = 3,
    aspect.ratio = 8
){
    # options(repr.plot.width = 50, repr.plot.height = 50)

    legend.title.format <- paste0(legend.title, "  - ")

    data.filtered <- subset(
        data, normal_class == normal.class, select = -normal_class
    )
    data.converted <- melt(
        data.filtered,
        id.vars = "experiment",
        variable.name = "class",
        value.name = "acc",
    )
    

    boxplot <- ggplot(
        data = data.converted,
        mapping = aes(
            x = class,
            y = acc,
            group = interaction(class, experiment),
        ),
    ) +
    geom_boxplot(aes(fill = experiment, color = experiment), lwd = .4) +

    # Group drawings per normal class
    facet_wrap(~class, scales = "free_x", nrow = 1) +

    # Rename Axes
    ylab("Precision @ Best AUC") +
    xlab("Class") +

    # Further Styling
    # scale_x_continuous(breaks = 0:9)
    # https://ggplot2-book.org/scales-guides.html
    labs(fill = legend.title.format, colour = legend.title.format) +
    ylim(lbound, 1) +
    ggtitle(
        TeX(
            sprintf(
                r'(%s (\textit{NC} = $%d$))',
                plot.title,
                normal.class
            )
        )
    )
    
    if (theme.publication) {
        boxplot <- boxplot +
                ThemePublication(base_family="LM Roman 10") +
                TransparentBackground() +
                ScalePublication(ngroups = ngroups) +
                ScalePublication2(ngroups = ngroups)
    }

    boxplot <- boxplot + theme(aspect.ratio = aspect.ratio) +
                         theme(plot.background = element_rect(fill = 'transparent', color=NA))
}

ClassTablePlot <- function(data, normal.class){
    options(repr.plot.width = 20, repr.plot.height = 2, digits = 3)
    data.filtered <- subset(
        data, normal_class == normal.class
    )

    data.reshaped <- melt(
        data.filtered,
        id.vars = c("experiment", "normal_class"),
        variable.name = "class",
        value.name = "acc",
    )
    
    set_flextable_defaults(digits = 3)
    table <- data.reshaped %>%
                group_by(experiment, class) %>%
                summarise_at(vars(acc), list(.mean = mean, .std = sd)) %>%
                tabulator(rows=c("experiment"), columns="class", `mean` = as_paragraph(.mean), `std` = as_paragraph(.std)) |>
                as_flextable()
    
    table.grob <- gen_grob(table)
    table.plot <- as.ggplot(table.grob)
    list(data.frame = table, plot = table.plot)
}

ClassTableLaTeX <- function(data, normal.class){
    options(repr.plot.width = 20, repr.plot.height = 2, digits = 3)
    data.filtered <- subset(
        data, normal_class == normal.class
    )

    data.reshaped <- melt(
        data.filtered,
        id.vars = c("experiment", "normal_class"),
        variable.name = "class",
        value.name = "acc",
    )

    table <- data.reshaped %>%
             group_by(experiment, class) %>%
             summarise_at(vars(acc), list(`mean` = mean, `sd` = sd)) %>%
             tabulator(
                rows=c("experiment"),
                columns="class",
                `$\\mu$` = as_paragraph(mean), 
                `$\\sigma$` = as_paragraph(sd)
             )

    keys.new <- colnames(table$data)[22:41]
    keys.old <- table$hidden_columns$col_keys

    table.data <- table$data[keys.old]
    colnames(table.data) <- keys.new

    reorder <- rep(1:10, rep_len(c(2), 10))
    reorder[seq(2, 20, 2)] <- 11:20

    table.data <- table.data[, reorder]
    table.format <- table.data %>% map(\(x) formatC(x, 3))
    table.data[names(table.format)] <- table.format

    row.names(table.data) <- table$data$experiment

    header.bottom <- colnames(table.data) %>% map(\(x) unlist(strsplit(x, "@"))[[1]])

    table.data <- rbind(header.bottom, table.data)
    row.names(table.data)[1] <- paste0("\\cmidrule(lr){", seq(2, 20, 2), "-", seq(3, 21, 2), "}", collapse="")
    row.names(table.data)[2] <- paste0("#midrule", row.names(table.data)[2], collapse="")

    header.top <- rep(1:10, rep_len(c(2), 10)) - 1
    header.top[seq(1, 20, 2)] <- as.character(header.top[seq(1, 20, 2)]) %>% map(\(x) paste("\\multicolumn{2}{c}{", x, "#"))
    header.top[seq(2, 20, 2)] <- rep("}", 10)

    table.data <- rbind(header.top, table.data)

    table.latex <- capture.output(
        print(
            xtable(table.data, align = "ccccccccccccccccccccc"),
            sanitize.text.function=function(x){x},
            comment = FALSE,
            include.colnames = FALSE,
            booktabs=TRUE
        )
    )
    
    table.latex <- gsub("# &", "", table.latex)
    table.latex <- gsub("\\\\midrule", "", table.latex)
    table.latex <- gsub("#midrule", "\\\\midrule ", table.latex)

    cat(table.latex,  sep="\n")
}


In [5]:
GroupedClassBoxPlot <- function(
    data,
    normal.class,
    plot.title = "",
    legend.title = "Experiment",
    theme.publication = TRUE,
    lbound = .7,
    ngroups = 3,
    aspect.ratio = 8
){
    options(repr.plot.width = 10, repr.plot.height = 8)

    legend.title.format <- paste0(legend.title, "  - ")

    data.filtered <- subset(
        data, normal_class == normal.class, select = -normal_class
    )
    data.converted <- melt(
        data.filtered,
        id.vars = "experiment",
        variable.name = "class",
        value.name = "acc",
    )

    data.converted$class <- ifelse(data.converted$class == normal.class, "Normal", "Anomalies")

    boxplot <- ggplot(
        data = data.converted,
        mapping = aes(
            x = class,
            y = acc,
            group = interaction(class, experiment),
        ),
    ) +
    geom_boxplot(aes(fill = experiment, color = experiment), lwd = .4) +

    # Group drawings per normal class
    facet_wrap(~class, scales = "free_x", nrow = 1) +

    # Rename Axes
    ylab("Precision @ Best AUC") +
    xlab("Class") +

    # Further Styling
    # scale_x_continuous(breaks = 0:9) +
    # https://ggplot2-book.org/scales-guides.html
    labs(fill = legend.title.format, colour = legend.title.format) +
    ylim(lbound, 1) +
    ggtitle(
        TeX(
            sprintf(
                r'(%s - \textit{normal class} $= %d$)',
                plot.title,
                normal.class
            )
        )
    )
    
    if (theme.publication) {
        boxplot <- boxplot +
                ThemePublication() +
                ScalePublication(ngroups = ngroups) +
                ScalePublication2(ngroups = ngroups) +
                TransparentBackground()
    }

    boxplot <- boxplot + theme(aspect.ratio = aspect.ratio) + theme(plot.background = element_rect(fill = 'transparent', color=NA))
}


In [6]:
BoxPlotFrom <- function(files, lbound, mapping, title, ordering, legend.title, name.conflict) {
    data = list()
    for (file in files) {
        data = append(data, list(read.csv(file)))
    }
    
    criterias <- c("experiment", "best_val_auc", "normal_class")
    
    binded <- NULL
    for (i in seq(1, length(data))) {
        data_ <- data[[i]]
        if (name.conflict) {
            data_$experiment <- paste0(data_$experiment[1], "_", i)
        }
        binded <- rbind(binded, data_[criterias])
    }

    recoded <- binded %>% mutate(
        experiment = recode(
            experiment,
            !!!mapping
        )
    )

    # recoded$experiment <- lapply(recoded$experiment, expression)
    # ordering <- lapply(ordering, expression)
    recoded$experiment <- factor(recoded$experiment, levels = ordering)
    # recoded <- recoded %>% mutate(
    #     experiment = expression(experiment)
    # )

    boxplot <- BoxPlot(
        data = recoded,
        plot.title = title,
        legend.title = legend.title,
        lbound = lbound,
        ngroups = length(data),
        labels = lapply(ordering, TeX)
    )
    
    list(boxplot = boxplot, data = recoded)
}

TablePlotFrom <- function(data, title) {
    table <- TablePlot(data, plot.title=title)

    print(
        xtable(table$data.frame, align = "cccc", digits = 3,),
        sanitize.text.function=function(x){x},
        comment = FALSE,
        include.rownames = FALSE,
        booktabs=TRUE,
    )

    table$plot
}

PerClassFrom <- function(files, lbound, mapping, title, ordering, legend.title, name.conflict) {
    data = list()
    for (file in files) {
        data = append(data, list(read.csv(file)))
    }
    
    columns = paste("best_val_acc_class_", 0:9, sep="")
    criterias <- c("experiment", "normal_class", columns)
    
    binded <- NULL
    for (i in seq(1, length(data))) {
        data_ <- data[[i]]
        if (name.conflict) {
            data_$experiment <- paste0(data_$experiment[1], "_", i)
        }
        binded <- rbind(binded, data_[criterias])
    }

    recoded <- binded %>% mutate(
        experiment = recode(
            experiment,
            !!!mapping
        )
    )
    
    recoded$experiment <- factor(recoded$experiment, levels = ordering)

    colnames(recoded)[3:12] = 0:9
    plots <- 0:9 |> map(
        \(x) ClassBoxPlot(
            recoded,
            plot.title = title,
            legend.title = legend.title,
            normal.class = x,
            lbound = lbound,
            ngroups = length(data),
            aspect.ratio = 10
        )
    )

    plot <- marrangeGrob(
        grobs=plots,
        nrow = 2,
        ncol = 5,
        top=NULL,
        layout_matrix = matrix(1:10, 2, 5, TRUE),
        newpage = FALSE,
        respect=TRUE
        )
    
    list(boxplot = plot, data = recoded)
}

In [7]:
Correlation <- function(file, average.file, compare.func, dataset, model) {
    data = list()
    data = append(data, list(read.csv(file)))

    averages <- read.table(average.file, sep=",")
    neg.scale <- compare.func == "Cosine Similarity"
    tmeans <- unlist(1:10 |> map(
        \(x) mean(c(t(averages[x]))[seq(1,10)])
    ))
    ranged <- NaN
    if (max(averages) > 1.01) {
        ranged <- max(averages)
        tmeans <- tmeans / max(averages)
        averages <- averages / max(averages)
    }
    subtitle <- paste(dataset, "-", compare.func)
    if (!is.na(ranged)) {
        subtitle <- TeX(sprintf(r'(%s \textit{(scaled by} $%#.2f$\textit{)})', subtitle, ranged))
    }
    subtitle2 <- paste(model, "- Mean Best AUC")
    
    criterias <- c("best_val_auc", "normal_class")
    
    binded <- NULL
    for (i in seq(1, length(data))) {
        data_ <- data[[i]]
        binded <- rbind(binded, data_[criterias])
    }

    averaged <- binded %>% group_by(normal_class) %>% summarise_at(vars(best_val_auc), list(mean_precision = mean))

    averaged <- averaged %>% mutate(measure = c(t(averages[11]))[1 + `normal_class`])
    averaged <- averaged %>% mutate(measure2 = tmeans[1 + normal_class])
    
    averaged["normal_class"] = lapply(averaged["normal_class"], as.factor)
    ggplot() +
    geom_point(data=averaged, shape=21, size=4, aes(
        x=measure,
        y=mean_precision,
        color = normal_class,
        fill = normal_class
    )) +
    geom_point(data=averaged, shape=3, size=1, show.legend = FALSE,
        aes(
        x=measure2,
        y=mean_precision,
        color = normal_class,
        fill = normal_class,
    )) +
    (if(neg.scale) xlim(-1, 1) else xlim(0, 1)) +
    ylim(0, 1) +
    ThemePublication(base_family="LM Roman 10") +
    ggtitle(paste0("AUC and Inter-Class Complexity Corr. (", compare.func,")")) +
    TransparentBackground(FALSE) +
    ScalePublication(10) + ScalePublication2(10) +
    labs(fill = "Normal Class -", color = "Normal Class -") +
    ylab(subtitle2) +
    xlab(subtitle)
}


In [8]:
MappedCorrelation <- function(data, normal.class, subtitle, subtitle2, neg.scale) {
    ggscatter(
        data,
        x="cosine_similarity",
        y="mean_precision",
        fill="class",
        color="class",
        shape=21,
        size=4,
        ylab=subtitle2,
        # add = "reg.line",
        xlim=if(neg.scale) c(-1, 1) else c(0, 1),
        ylim=c(0, 1)
    ) +
    xlab(subtitle) +
    facet_wrap(~normal_class, scales = "free_x", nrow = 2) +
    ThemePublication(base_family="LM Roman 10") +
    ggtitle(
        TeX(
            sprintf(
                r'(%s (\textit{NC} = $%d$))',
                "Neg. Precisions and Inter-Class Complexity Corr.",
                normal.class
            )
        )
    ) +
    TransparentBackground(FALSE) +
    ScalePublication3(normal.class) + ScalePublication4(normal.class) +
    labs(fill = "Anomalies Original Classes -", color = "Anomalies Original Classes -")
}

CorrelationPerClass <- function(file, average.file, compare.func, average, dataset, model) {
    data = list()
    data = append(data, list(read.csv(file)))

    averages <- read.table(average.file, sep=",")
    
    neg.scale <- compare.func == "Cosine Similarity"
    ranged <- NaN
    if (max(averages) > 1.01) {
        ranged <- max(averages)
        averages <- averages / max(averages)
    }
    subtitle <- paste(dataset, "-", compare.func)
    if (!is.na(ranged)) {
        subtitle <- TeX(sprintf(r'(%s \textit{(scaled by} $%#.2f$\textit{)})', subtitle, ranged))
    }
    subtitle2 <- paste(model, "- Mean Negative Precision @ Best AUC")
    
    criterias <- c("best_val_auc", "normal_class", paste0("best_val_acc_class_", seq(0, 9)))
    
    binded <- NULL
    for (i in seq(1, length(data))) {
        data_ <- data[[i]]
        binded <- rbind(binded, data_[criterias])
    }

    colnames(binded) <- c('best_val_auc', 'normal_class', seq(0, 9))

    rearranged <- melt(
        binded,
        id = c("best_val_auc", "normal_class"),
        variable.name = "class",
        value.name = "best_auc"
    )
    rearranged["class"] = lapply(rearranged["class"], function(x) as.integer(x) - 1)
    # rearranged
    # averaged <- rearranged %>% group_by(normal_class, class) %>% summarise_at(vars(best_auc), list(mean_precision = mean))

    if (average) {
        averaged <- rearranged %>% group_by(normal_class, class) %>% summarise_at(vars(best_auc), list(mean_precision = mean))
    }
    else {
        averaged <- rearranged[c("normal_class", "class", "best_auc")]
        averaged <- averaged %>% rename("mean_precision" = "best_auc")
        averaged <- arrange(averaged, normal_class, class)
    }

    averaged <- averaged %>% mutate(cosine_similarity = c(t(averages[1 + `normal_class`,]))[1 + `class`])
    averaged <- subset(averaged, `normal_class` != `class`)

    averaged["normal_class"] = lapply(averaged["normal_class"], as.factor)
    averaged["class"] = lapply(averaged["class"], as.factor)
    # list(averages = averages, averaged = averaged)

    plots <- 0:9 |> map(
        \(x) MappedCorrelation(
            averaged[averaged$normal_class == x,],
            x,
            subtitle,
            subtitle2,
            neg.scale
        )
    )

    plot <- marrangeGrob(
        grobs=plots,
        nrow = 2,
        ncol = 5,
        top=NULL,
        layout_matrix = matrix(1:10, 2, 5, TRUE),
        newpage = FALSE,
        respect=TRUE
        )
}

In [9]:
Correlations <- function(file, average.files, compare.funcs, dataset, model) {
    plots <- 1:length(average.files) |> map(
        \(x) Correlation(
            file,
            average.files[x],
            compare.funcs[x],
            dataset,
            model
        )
    )

    plot <- marrangeGrob(
        grobs=plots,
        nrow = 1,
        ncol = length(average.files),
        top=NULL,
        # layout_matrix = matrix(1:10, 2, 5, TRUE),
        newpage = FALSE,
        respect=TRUE
        )
}

In [10]:
AnoCorrelation <- function(file, average.file, compare.func, dataset, model) {
    data = list()
    data = append(data, list(read.csv(file)))

    averages <- read.table(average.file, sep=",")
    neg.scale <- compare.func == "Cosine Similarity"
    diag <- unlist(1:10 |> map(
        \(x) c(t(averages[x]))[x])
    )

    subtitle <- paste(dataset, "-", "Pseudo-Anomalies", "-", compare.func)
    subtitle2 <- paste(model, "- Mean Best AUC")
    
    criterias <- c("best_val_auc", "normal_class")
    
    binded <- NULL
    for (i in seq(1, length(data))) {
        data_ <- data[[i]]
        binded <- rbind(binded, data_[criterias])
    }

    averaged <- binded %>% group_by(normal_class) %>% summarise_at(vars(best_val_auc), list(mean_precision = mean))

    averaged <- averaged %>% mutate(measure = diag[1 + normal_class])
    
    averaged["normal_class"] = lapply(averaged["normal_class"], as.factor)
    ggplot() +
    geom_point(data=averaged, shape=21, size=4, aes(
        x=measure,
        y=mean_precision,
        color = normal_class,
        fill = normal_class
    )) +
    (if(neg.scale) xlim(-1, 1) else xlim(0, 1)) +
    ylim(0, 1) +
    ThemePublication(base_family="LM Roman 10") +
    ggtitle(paste0("AUC and Pseudo-Ano. vs. Normal Data Corr. (", compare.func,")")) +
    TransparentBackground(FALSE) +
    ScalePublication(10) + ScalePublication2(10) +
    labs(fill = "Normal Class -", color = "Normal Class -") +
    ylab(subtitle2) +
    xlab(subtitle)
}

# WACV

In [11]:
files <- c("../log/xp9/rcsplad_lv2_t05.csv", "../log/xp9/rcsplad_lv2_t05_l10.csv", "../log/xp9/rcsplad_lv2_t05_l20.csv")
mapping <- c(rcsplad_lv2_t05 = "PLUME - \\lambda=5", rcsplad_lv2_t05_l10 = "PLUME - \\lambda=10", rcsplad_lv2_t05_l20 = "PLUME - \\lambda=20")
ordering <- c("PLUME - \\lambda=5", "PLUME - \\lambda=10", "PLUME - \\lambda=20")
lbound <- .4
title <- "CIFAR10 - \\tau=0.5"
legend.title <- "Experiment"
save.file <- "lambda_xp"
name.conflict <- FALSE

# Plot Here

In [ ]:
bp.result <- BoxPlotFrom(files, lbound, mapping, title, ordering, legend.title, name.conflict)

options(repr.plot.width = NULL, repr.plot.height = NULL)
bp.result$boxplot <- bp.result$boxplot + guides(fill = guide_legend(nrow=2), color = guide_legend(nrow=2))
bp.result$boxplot

In [ ]:
FancyTable <- function(data){
    options(digits = 3)

    data$best_val_auc <- data$best_val_auc * 100

    table <- data %>%
             group_by(experiment, normal_class) %>%
             summarise_at(vars(best_val_auc), list(`mean` = mean, `sd` = sd)) %>%
             tabulator(
                rows="experiment",
                columns="normal_class",
                `$\\mu$` = as_paragraph(mean), 
                `$\\sigma$` = as_paragraph(sd)
             )

    table2 <- data %>%
             group_by(experiment) %>%
             summarise_at(vars(best_val_auc), list(`mean` = mean))
    table2 <- table2 %>% map(\(x) formatC(x, digits=1, format="f"))

    table3 <- data %>%
             group_by(experiment, normal_class) %>%
             summarise_at(vars(best_val_auc), list(`max` = max)) %>%
             tabulator(
                rows="experiment",
                columns="normal_class",
                `best` = as_paragraph(max)
             )
    table3.data <- table3$data[1:11]

    table4 <- rowMeans(table3.data[, 2:11])
    table4 <- table4  %>% unlist
    table4 <- cbind(EMPTY = NA, table3.data[,2:11], best_mean=table4)
    row.names(table4) <- table3.data$experiment
    
    table$data <- table$data[1:41]
    keys.new <- colnames(table$data)[22:41]
    keys.old <- table$hidden_columns$col_keys

    table.data <- table$data[keys.old]
    colnames(table.data) <- keys.new

    reorder <- rep(1:10, rep_len(c(2), 10))
    reorder[seq(2, 20, 2)] <- 11:20

    table.data <- table.data[, reorder]

    mus <- grepl("mu", colnames(table.data))
    sigmas <- grepl("sigma", colnames(table.data))

    table.format <- table.data %>% map(\(x) formatC(x, digits=1, format="f"))
    table.format[sigmas] <- table.data[sigmas] %>% map(\(x) formatC(x, digits=1, format="f"))
    table.data[names(table.format)] <- table.format

    row.names(table.data) <- table$data$experiment

    for (i in seq(from=1, to=length(table.data) - 1, by=2)) {
        table.data[i] <- do.call(paste, c(table.data[c(i, i + 1)], sep="\\stdv{"))
        table.data[i] <- do.call(paste, c(table.data[i], "}", sep=""))
    }

    table.data <- table.data[mus]
    table.data$mean <- table2$mean
    table.data <- cbind(EMPTY = NA, table.data)

    # return(table.data)
    table.latex <- capture.output(
        print(
            xtable(table.data, align = "ccccccccccccc"),
            sanitize.text.function=function(x){x},
            comment = FALSE,
            include.colnames = FALSE,
            booktabs=TRUE
        )
    )
    
    table.latex <- gsub("\\\\midrule", "", table.latex)

    table4[2:11] <- table4[2:11]  %>% map(\(x) formatC(x, digits=1, format="f"))
    cat(table.latex,  sep="\n")
    table.latex2 <- capture.output(
        print(
            xtable(table4, align = "ccccccccccccc"),
            sanitize.text.function=function(x){x},
            comment = FALSE,
            include.colnames = FALSE,
            booktabs=TRUE
        )
    )

    cat(table.latex2,  sep="\n")
}

FancyTable(bp.result$data)

In [24]:
ggsave(paste0(base.dir, save.file, ".pdf"), bp.result$boxplot)

Saving 6.67 x 6.67 in image


Warning message:
“Removed 1 rows containing non-finite values (`stat_boxplot()`).”
